In [1]:
%load_ext tensorboard

import tensorflow as tf
import keras
from keras import layers
import yaml, datetime
from tensorboard.plugins.hparams import api as hp

2024-01-06 16:25:08.363823: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-06 16:25:11.625369: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([2, 4, 8, 16]))
HP_IMG_DIM_SIZE = hp.HParam('img_dim_size', hp.Discrete([64, 128, 256]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd', 'rmsprop']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_BATCH_SIZE, HP_IMG_DIM_SIZE, HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

with open('config.yml', 'r') as file:
    paths = yaml.safe_load(file)

data_dir = paths['paths']['categories_path']

2024-01-06 16:25:20.357552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-06 16:25:20.635605: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-06 16:25:20.636785: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
def train_test_model(hparams):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(hparams[HP_IMG_DIM_SIZE], hparams[HP_IMG_DIM_SIZE]),
        batch_size=hparams[HP_BATCH_SIZE])

    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(hparams[HP_IMG_DIM_SIZE], hparams[HP_IMG_DIM_SIZE]),
        batch_size=hparams[HP_BATCH_SIZE])

    val_batches = tf.data.experimental.cardinality(val_ds)
    test_ds = val_ds.take((2*val_batches) // 3)
    val_ds = val_ds.skip((2*val_batches) // 3)

    data_augmentation = keras.Sequential(
        [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
        ]
    )

    input = keras.Input(shape=(hparams[HP_IMG_DIM_SIZE], hparams[HP_IMG_DIM_SIZE], 3))
    x = data_augmentation(input)
    x = layers.Rescaling(1./255)(x)
    x = layers.Conv2D(filters=16, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2,2))(x)
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2,2))(x)
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2,2))(x)
    x = layers.Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2,2))(x)
    x = layers.Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2,2))(x)
    x = layers.Conv2D(filters=512, kernel_size=(3, 3), padding='same', activation='relu')(x)
    x = layers.Flatten()(x)
    output = layers.Dense(7, activation='softmax')(x)
    model = keras.Model(inputs=input, outputs=output)

    model.compile(loss='sparse_categorical_crossentropy', optimizer=hparams[HP_OPTIMIZER], metrics=['accuracy'])

    model.fit(train_ds, 
              epochs=50, 
              validation_data=val_ds)
    _, accuracy = model.evaluate(test_ds)
    return accuracy

In [8]:
def run(run_dir, hparams):
  with tf.summary.create_file_writer(run_dir).as_default():
    hp.hparams(hparams)  
    accuracy = train_test_model(hparams)
    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)


In [9]:
session_num = 0

for batch_size in HP_BATCH_SIZE.domain.values:
  for img_dim_size in HP_IMG_DIM_SIZE.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
      hparams = {
          HP_BATCH_SIZE: batch_size,
          HP_IMG_DIM_SIZE: img_dim_size,
          HP_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('--- Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      run('logs/hparam_tuning/' + run_name, hparams)
      session_num += 1


230/230 [==============================] - 11s 47ms/step - loss: 1.4947 - accuracy: 0.4115 - val_loss: 1.9219 - val_accuracy: 0.3539
Epoch 26/50
 99/230 [===========>..................] - ETA: 4s - loss: 1.4854 - accuracy: 0.4104

2024-01-07 01:18:57.634553: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: extra compressed data


230/230 [==============================] - 9s 39ms/step - loss: 1.4896 - accuracy: 0.4110 - val_loss: 1.5963 - val_accuracy: 0.3864
Epoch 27/50
 99/230 [===========>..................] - ETA: 6s - loss: 1.4628 - accuracy: 0.4217

2024-01-07 01:19:08.107487: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: extra compressed data


230/230 [==============================] - 12s 50ms/step - loss: 1.4752 - accuracy: 0.4254 - val_loss: 1.6383 - val_accuracy: 0.3604
Epoch 28/50
 98/230 [===========>..................] - ETA: 4s - loss: 1.4685 - accuracy: 0.4216

...

230/230 [==============================] - 17s 72ms/step - loss: 1.0943 - accuracy: 0.5803 - val_loss: 1.7056 - val_accuracy: 0.4221
Epoch 50/50
 91/230 [==========>...................] - ETA: 9s - loss: 1.0810 - accuracy: 0.5838

2024-01-07 02:06:08.291522: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: extra compressed data


230/230 [==============================] - 17s 73ms/step - loss: 1.0725 - accuracy: 0.5918 - val_loss: 1.8007 - val_accuracy: 0.3864


2024-01-07 02:06:19.065777: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [916]
	 [[{{node Placeholder/_4}}]]
2024-01-07 02:06:19.067679: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [916]
	 [[{{node Placeholder/_0}}]]


38/38 [==============================] - 2s 45ms/step - loss: 1.7183 - accuracy: 0.3997


In [11]:
%tensorboard --logdir logs/hparam_tuning

Reusing TensorBoard on port 6006 (pid 99189), started 0:00:06 ago. (Use '!kill 99189' to kill it.)